This script creates a short test video (and associated start/end frames) from a SINGLE complete gameplay video. It requires a begining and an ending X_screen_total value from the event file.

In [ ]:
import os
import re
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

# Import the cleaned event file
df = pd.read_csv('../data/datasets/combined_events_level-1_clean_outset_vid.csv')

# Choose a single video file (.mp4)
video_file_path = '../output/videos/videos_full/sub-01_ses-003_task-shinobi_run-01_level-1_rep-01.mp4'

# Extract the base name of the video file (without path and extension)
video_file_name = os.path.basename(video_file_path)
video_file_name_no_ext = os.path.splitext(video_file_name)[0]

# Parse the video file name to extract subject, session, run, level, and repetition
pattern = r'(?P<subject>sub-[^_]+)_(?P<session>ses-[^_]+)_task-[^_]+_(?P<run>run-[^_]+)_(?P<level>level-[^_]+)_(?P<repetition>rep-[^_]+)'
match = re.match(pattern, video_file_name_no_ext)

if not match:
    raise ValueError("Video file name does not match the expected pattern")

subject = match.group('subject')
session = match.group('session')
run = match.group('run')
level = match.group('level')
repetition = match.group('repetition')

# Print extracted values to verify
print(f"Subject: {subject}")
print(f"Session: {session}")
print(f"Run: {run}")
print(f"Level: {level}")
print(f"Repetition: {repetition}")

# Filter the DataFrame for the specified session, run, and repetition
df_filtered = df[(df['level'] == level) &
                 (df['session'] == session) & 
                 (df['run'] == run) & 
                 (df['repetition'] == repetition)]

# Load the video file with moviepy
video = VideoFileClip(video_file_path)

# Get frames per second (fps) from the video
fps = video.fps
print(f"Processing {video_file_path} with FPS: {fps}")

# Define the start and end x_screen_total values
start_x_screen_total = 4550  # <-- Change this value to adjust the START of the video
end_x_screen_total = 5150    # <-- Change this value to adjust the END of the video

# Get start and end onsets from the dataframe
start_onset = float(df_filtered[df_filtered['X_screen_total'] >= start_x_screen_total]['onset_vid'].values[0])
end_onset = float(df_filtered[df_filtered['X_screen_total'] >= end_x_screen_total]['onset_vid'].values[0])

# Cut the video using the onset times
video_cut = video.subclip(start_onset, end_onset)

# Write the output video file with audio
output_video_base_path = '../data/test_data'
output_video_file_name = f'{subject}_{session}_task-shinobi_{run}_{level}_{repetition}__TEST_SHORT.mp4'
output_video_file_path = os.path.join(output_video_base_path, output_video_file_name)

video_cut.write_videofile(output_video_file_path, codec='libx264', audio_codec='aac', fps=fps)

# Print video information
fps = video.fps
n_frames = int(video_cut.reader.nframes)
print(f'Video start time (s): {start_onset}')
print(f'Video end time (s): {end_onset}')
print(f'Total time (s): {n_frames / fps:.3f}')
print(f"Video frames per second (fps): {fps}")
print(f'Total frame number: {n_frames}')

cap = cv2.VideoCapture(video_file_path)

# Get frames from the video file
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

# Release the video capture object
cap.release()

# Find frame indices for the start and end onsets
frame_start = int(df_filtered[(df_filtered['onset_vid'] == start_onset)]['frame'].values[0])   # Choose the start frame
frame_end = int(df_filtered[(df_filtered['onset_vid'] == end_onset)]['frame'].values[0])     # Choose the end frame

# Save the start frame
start_frame_path = os.path.join(output_video_base_path, f'{video_file_name_no_ext}_start_frame.png')
cv2.imwrite(start_frame_path, frames[frame_start])
print(f'Saved start frame at: {start_frame_path}')

# Save the end frame
end_frame_path = os.path.join(output_video_base_path, f'{video_file_name_no_ext}_end_frame.png')
cv2.imwrite(end_frame_path, frames[frame_end])
print(f'Saved end frame at: {end_frame_path}')

# Plot the start and end frames
plt.imshow(cv2.cvtColor(frames[frame_start], cv2.COLOR_BGR2RGB))
plt.title(f'Start Frame (Frame #{frame_start}, Onset {start_onset}s)')
plt.show()

plt.imshow(cv2.cvtColor(frames[frame_end], cv2.COLOR_BGR2RGB))
plt.title(f'End Frame (Frame #{frame_end}, Onset {end_onset}s)')
plt.show()